In [ ]:
import pandas as pd
import gspread
from google.oauth2.service_account import Credentials
from gspread.exceptions import SpreadsheetNotFound, APIError
import schedule
import time
from datetime import datetime

SCOPES = ["https://www.googleapis.com/auth/spreadsheets", "https://www.googleapis.com/auth/drive"]
SERVICE_ACCOUNT_FILE = "csv-automation-project-475202-9c0b137d3f4a.json"
creds = Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)
client = gspread.authorize(creds)
SHEET_NAME = "leads"

def upload_leads_to_sheets():
    print("\n🚀 Starting automation:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

    data = {
        "Name": ["Rana Ali", "Omar Hassan"],
        "Email": ["rana@example.com", "omar@gmail.com"],
        "Message": ["Interested in your course", "Please send me more info"]
    }
    df = pd.DataFrame(data)
    csv_file = "leads.csv"
    df.to_csv(csv_file, index=False)
    print("✅ CSV file 'leads.csv' created successfully!")

    try:
        spreadsheet = client.open(SHEET_NAME)
        print(f"Found existing Google Sheet: '{SHEET_NAME}'")
    except SpreadsheetNotFound:
        try:
            print(f" Sheet '{SHEET_NAME}' not found. Creating new one...")
            spreadsheet = client.create(SHEET_NAME)
            user_email = "rana3lii09@gmail.com"  
            spreadsheet.share(user_email, perm_type='user', role='writer')
            print(f" New sheet created and shared with {user_email}")
        except APIError as e:
            print(" Couldn't create sheet automatically. Check Drive quota or share permissions.")
            return

    try:
        sheet = spreadsheet.sheet1
        data_to_upload = [df.columns.values.tolist()] + df.values.tolist()
        sheet.append_rows(data_to_upload)
        print(" CSV uploaded successfully to Google Sheets!")
    except Exception as e:
        print(f" Upload failed: {e}")

schedule.every().day.at("09:00").do(upload_leads_to_sheets)  
print(" Daily Google Sheets automation is running...")
print("It will automatically upload the data every day at 9:00 AM.")

while True:
    schedule.run_pending()
    time.sleep(60)


 Daily Google Sheets automation is running...
It will automatically upload the data every day at 9:00 AM.
